In [6]:
import pandas as pd
!pip install river

In [2]:
dataset = pd.read_csv("finance.csv")

In [3]:
dataset.head()

,time,value
0,2022-07-22 10:28:35,0.025808
1,2022-07-22 10:33:35,0.616177
2,2022-07-22 10:38:36,0.616177
3,2022-07-22 10:43:37,1.100507
4,2022-07-22 10:48:36,-0.044577


In [20]:
params = {'converters': {'value': float},'parse_dates': {'time': "%Y-%m-%d %H:%M:%S"}}




dataset = dict()
from river import stream
for x, y in stream.iter_csv('finance.csv', target = 'value', **params):
    print(x,y)

{'time': datetime.datetime(2022, 7, 22, 10, 28, 35)} 0.025808099608183845
{'time': datetime.datetime(2022, 7, 22, 10, 33, 35)} 0.6161768921730953
{'time': datetime.datetime(2022, 7, 22, 10, 38, 36)} 0.6161768921730953
{'time': datetime.datetime(2022, 7, 22, 10, 43, 37)} 1.1005073910122758
{'time': datetime.datetime(2022, 7, 22, 10, 48, 36)} -0.0445770393033232
{'time': datetime.datetime(2022, 7, 22, 10, 53, 35)} -0.0445770393033232
{'time': datetime.datetime(2022, 7, 22, 10, 58, 35)} 0.6991721637759272
{'time': datetime.datetime(2022, 7, 22, 11, 3, 35)} 0.6161768921730953
{'time': datetime.datetime(2022, 7, 22, 11, 8, 35)} 0.6161768921730953
{'time': datetime.datetime(2022, 7, 22, 11, 18, 37)} 1.4841174003594793
{'time': datetime.datetime(2022, 7, 22, 11, 23, 35)} 1.4841174003594793
{'time': datetime.datetime(2022, 7, 22, 11, 28, 36)} 0.9153650878428139
{'time': datetime.datetime(2022, 7, 22, 11, 33, 34)} 1.4201790692709686
{'time': datetime.datetime(2022, 7, 22, 11, 43, 37)} 1.4201790

In [16]:
from river import compose
from river import linear_model
from river import metrics
from river import evaluate
from river import preprocessing

In [24]:
import math
import datetime
hour_list = [str(i) for i in range (0,24)]
minute_list = [f'{str(i)} min' for i in range (0, 60)]
def get_hour_distances(x):
    return {
        hour_list[hour]: math.exp(-(x['time'].hour - hour) ** 2)
        for hour in range(0, 24)
    }
def get_minute_distances(x):
    return {
        minute_list[minute]: math.exp(-(x['time'].minute - minute) ** 2)
        for minute in range(0, 60)
    } 
def get_date_progress(x):
    return {'date': x['time'].toordinal() - datetime.datetime(2022, 1, 1, 0, 0).toordinal()}

In [27]:
model = compose.Pipeline(
    ('features', compose.TransformerUnion(
        ('date_progress', compose.FuncTransformer(get_date_progress))
#         ('hour', compose.FuncTransformer(get_hour_distances))
#         ('minute', compose.FuncTransformer(get_minute_distances))
    )),
    ('scale', preprocessing.StandardScaler()),
    ('lin_reg', linear_model.LinearRegression(intercept_lr=0))
)
metric = metrics.MAE() + metrics.R2()
model = preprocessing.TargetStandardScaler(regressor=model)
evaluate.progressive_val_score(stream.iter_csv('finance.csv', target = 'value', **params), model, metric, print_every=100)

[100] MAE: 0.89063, R2: 0.112435
[200] MAE: 0.774902, R2: 0.14066
[300] MAE: 0.764956, R2: 0.104904
[400] MAE: 0.768909, R2: 0.080444
[500] MAE: 0.654881, R2: 0.138635
[600] MAE: 0.571166, R2: 0.144799
[700] MAE: 0.495429, R2: 0.145226
[800] MAE: 0.444356, R2: 0.162071
[900] MAE: 0.423259, R2: 0.169029
[1,000] MAE: 0.423669, R2: 0.154445


MAE: 0.423669, R2: 0.154445